In [1]:
import import_ipynb
import betfirst
import unibet
import arbitrage
import bwin

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

def get_page_html(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url) 
    time.sleep(5)
    html = driver.page_source    
    driver.quit()
    return html

In [3]:
from bs4 import BeautifulSoup 
import pandas as pd

def extract_matches_betfirst(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    match_data = []
    
    # Trouver tous les matchs
    matches = soup.find_all('div', class_='rj-ev-list__ev-card')
    
    for match in matches:
        try:
            match_info = {}
            
            # Extraire les équipes
            teams = match.find_all('span', class_='rj-ev-list__name-text')
            if len(teams) >= 2:
                match_info['Home_Team'] = teams[0].text.strip()
                match_info['Away_Team'] = teams[1].text.strip()
            
            # Extraire les cotes
            odds = match.find_all('span', class_='rj-ev-list__bet-btn__odd')
            if len(odds) >= 2:
                match_info['Home_Odds'] = float(odds[0].text.strip())
                match_info['Away_Odds'] = float(odds[1].text.strip())
                
            # Ajouter le match seulement si on a toutes les infos
            if len(match_info) == 4:
                match_data.append(match_info)
                
        except Exception as e:
            print(f"Erreur lors de l'extraction d'un match: {e}")
            continue

    df = pd.DataFrame(match_data)
    
    return df

In [4]:
def main():
   betfirst_url = "https://betfirst.dhnet.be/en/basketball/united-states-nba/"
   unibet_url = "https://fr.unibetsports.be/betting/sports/filter/basketball/all/matches" 
   bwin_url = "https://www.bwin.be/en/sports/basketball-7/betting/north-america-9/nba-6004"

   betfirst_html = get_page_html(betfirst_url)
   unibet_html = get_page_html(unibet_url)
   bwin_html = get_page_html(bwin_url)

   # Récupérer tous les DataFrames
   dataframes = {
       'Betfirst': extract_matches_betfirst(betfirst_html),
       'Unibet': unibet.extract_matches_unibet(unibet_html),
       'Bwin': bwin.extract_moneyline_bwin(bwin_html)
   }

   for name, df in dataframes.items():
        if not df.empty:
            filename = f"{name}_odds_{pd.Timestamp.now().strftime('%Y%m%d_%H%M')}.csv"
            df.to_csv(filename, index=False)
            print(f"\n{name} : données sauvegardées dans {filename}")
        else:
            print(f"\n{name} : aucune donnée à sauvegarder")
   # Comparer chaque paire de bookmakers
   bookmakers = list(dataframes.keys())
   for i in range(len(bookmakers)):
       for j in range(i + 1, len(bookmakers)):
           book1 = bookmakers[i]
           book2 = bookmakers[j]
           print(f"\nAnalyse d'arbitrage entre {book1} et {book2}:")
           arbitrage.analyze_arbitrage(dataframes[book1], dataframes[book2])

if __name__ == "__main__":
   main()

Nombre total de matchs extraits : 6

Betfirst : données sauvegardées dans Betfirst_odds_20250105_1909.csv

Unibet : données sauvegardées dans Unibet_odds_20250105_1909.csv

Bwin : données sauvegardées dans Bwin_odds_20250105_1909.csv

Analyse d'arbitrage entre Betfirst et Unibet:
OKC Thunder BOS Celtics
Home_Team    Oklahoma City Thunder
Away_Team           Boston Celtics
Home_Odds                      1.8
Away_Odds                     2.07
Name: 0, dtype: object

Match: OKC Thunder vs BOS Celtics
Match trouvé dans df2: Oklahoma City Thunder vs Boston Celtics
Valeurs df1: Home_Odds=1.8, Away_Odds=2.15
Valeurs df2: Home_Odds=1.8, Away_Odds=2.07
Arbitrage Home/Away: -3.86%
Arbitrage Away/Home: -2.07%
CLE Cavaliers CHA Hornets
Home_Team    Cleveland Cavaliers
Away_Team      Charlotte Hornets
Home_Odds                   1.08
Away_Odds                    8.5
Name: 2, dtype: object

Match: CLE Cavaliers vs CHA Hornets
Match trouvé dans df2: Cleveland Cavaliers vs Charlotte Hornets
Valeurs df